<a href="https://colab.research.google.com/github/sburkley/CS-419/blob/master/projects/Sally's_Bach_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sally Burkley, 10-6-2020

In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree

#do not use choral_ID or event_number for training/testing
chords = pd.read_csv("https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip")
chords.set_index('choral_ID', inplace=True)
chords

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
choral_ID,,,,,,,,,,,,,,,,
000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M


In [98]:
chords = chords.drop("event_number", axis = 1)
one_hot = pd.get_dummies(chords['bass'], prefix=["bass"])
chords = chords.drop('bass', axis=1)
chords = chords.join(one_hot)


In [99]:
noteArray = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
for i in noteArray:
  chords[i] = chords[i].apply(lambda x: 1 if x == 'YES' else 0)
chords

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,meter,chord_label,['bass']_A,['bass']_A#,['bass']_Ab,['bass']_B,['bass']_Bb,['bass']_C,['bass']_C#,['bass']_D,['bass']_D#,['bass']_Db,['bass']_E,['bass']_Eb,['bass']_F,['bass']_F#,['bass']_G,['bass']_G#
choral_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000106b_,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,0,0,0,0,0,1,0,0,0,1,0,0,4,F_M,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,0,0,0,0,0,1,0,0,0,1,0,0,4,F_M,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,0,0,0,0,0,1,0,0,0,1,0,0,4,F_M,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [116]:
#chords_train, chords_test = train_test_split(chords, test_size = 0.2)
#chords_trainLabel = chords_train['chord_label']
#chords_trainFeatures = chords_train.drop('chord_label', axis=1)
#chords_testLabel = chords_test['chord_label']
#chords_testFeatures = chords_test.drop('chord_label', axis=1)

dtc = tree.DecisionTreeClassifier(criterion='entropy') #made the classifer

chords_Features = chords.drop('chord_label', axis = 1)
chords_Label = chords['chord_label']
chords_trainFeatures, chords_testFeatures, chords_trainLabel, chords_testLabel = train_test_split(chords_Features, chords_Label, test_size = 0.2, random_state=42)

dtc.fit(chords_trainFeatures, chords_trainLabel)
scores = cross_val_score(dtc, chords_testFeatures, chords_testLabel, cv=10)
print(scores)
print("The average CROSS VALUE accuracy is %5.2f" % (scores.mean()))

[0.81121683 0.80420631 0.81447171 0.81305291 0.80944829 0.81312077
 0.80928136 0.8086971  0.80903097 0.81153493]
The average CROSS VALUE accuracy is  0.81


In [117]:
from sklearn.metrics import accuracy_score
dtc.fit(chords_trainFeatures, chords_trainLabel)
chords_predictions = dtc.predict(chords_testFeatures)
print(chords_predictions)
accuracy = accuracy_score(chords_testLabel, chords_predictions, normalize=True, sample_weight=None)
print("The accuracy is %5.2f" % accuracy)

['B_M' 'G_m' 'F#M' ... 'A_M' 'G_M' 'D_m7']
The accuracy is  0.82


In [118]:
comparison = pd.DataFrame(chords_testLabel)
comparison['prediction'] = chords_predictions
comparison

,chord_label,prediction
choral_ID,,
013906b_,B_M,B_M
000206b_,G_m,G_m
000408b_,F#M,F#M
015301b_,C#d7,C#d7
014007b_,BbM,BbM
...,...,...
000907b_,E_M,E_M
003006b_,A_M6,A_M6
002908ch,A_M,A_M
